In [1]:
!pip install chromadb # python client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/4

In [1]:
import chromadb
import json

In [2]:
with open('/content/Tabela_NCM_Vigente_20260121.json', 'r') as file:
  ncm_data = json.load(file)

In [3]:
ncm_dic = {}
for ncm in ncm_data["Nomenclaturas"]:
  ncm_dic[ncm['Codigo']] = ncm['Descricao']

In [4]:
ncm_leaf = [x for x in ncm_dic.keys() if len(x)==10 and int(x[:2]) < 14]

In [5]:
ncm_dic_fmt = {}
for x in ncm_leaf:
  prefixo_chave = f"{x[:2]}.{x[2:4]}"
  prefixo_prod = ncm_dic[prefixo_chave] if prefixo_chave in ncm_dic.keys() else ""
  prefixo_chave = f"{x[:2]}.{x[2:4]}.{x[5:6]}"
  prefixo_prod += ncm_dic[prefixo_chave] if prefixo_chave in ncm_dic.keys() else ""
  prefixo_chave = f"{x[:2]}.{x[2:4]}.{x[5:7]}"
  prefixo_prod += ncm_dic[prefixo_chave] if prefixo_chave in ncm_dic.keys() else ""
  prefixo_chave = f"{x[:2]}.{x[2:4]}.{x[5:7]}.{x[8:9]}"
  prefixo_prod += ncm_dic[prefixo_chave] if prefixo_chave in ncm_dic.keys() else ""
  prefixo_prod += ncm_dic[x] if x in ncm_dic.keys() else ""
  ncm_dic_fmt[x] = prefixo_prod

952

In [7]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="ncm")

collection.add(
    documents=list(ncm_dic_fmt.values()),
    ids=list(ncm_dic_fmt.keys())
)

In [16]:
results = collection.query(
    query_texts=[
        "arroz com casca parboilizado"
    ],
    n_results=5
)

print(results)

{'ids': [['1006.10.91', '1006.20.10', '1006.10.92', '1006.20.20', '1006.40.00']], 'embeddings': None, 'documents': [['Arroz.Parboilizado', 'Arroz.Parboilizado', 'Arroz.Não parboilizado', 'Arroz.Não parboilizado', 'Arroz.- Arroz quebrado (Trinca de arroz*)']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None, None, None, None]], 'distances': [[0.3009647727012634, 0.3009647727012634, 0.345161110162735, 0.345161110162735, 0.68097984790802]]}
